<a href="https://colab.research.google.com/github/andrkech/GENERATIVE-METHODS-IN-GENOMICS/blob/main/FASTQ_DataLoader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf

BATCH_SIZE = 32
FASTQ_DIR = "/content/fastq_dataset/"

train_dataset = data_generator.flow_from_directory(fastq_dir,
                                                   target_size)

class FASTQ_DataLoader(seq_length):
  def __inti__(self, data_dir, batch_size, seq_length):
    self.data_dir = data_dir
    self.batch_size = batch_size
    self.seq_length = seq_length

    self.fastq_files = [f for f in os.listdir(data_dir)]

    def load_batch(self):
      batch_files = np.random.choice(self.fastq_files, size=self.batch_size, replace=False)
      batch_sequences = []
      bath_quality_scores = []

      for file in batch_files:
        file_path = os.path.join(self.data_dir, file)
        with open(file_path, 'r') as f:
          seq, qs = self.extract_seq_and_qs(f)

        seq = self.pad_or_truncate(seq, self.seq_length)

        batch_sequences.apppend(seq)
        batch_quality_quality_scores(qs)

      return np.array(batch_sequences), np.array(batch_quality_scores)

      def extract_seq_and_qs(self, fastq_file):
        pass

      def pad_or_truncate(self, seq, target_length):
        pass

DATA_DIR = ""
BATCH_SIZE = 16
dna_loader = FASTQLoader(DATA_DIR, BATCH_SIZE)

batch_sequences, batch_quality_scores = dna_loader.load_batch()